In [1]:
from profiler.core import *

In [2]:
pf = Profiler(workers=2)

In [3]:
pf.session.load_data(name='hospital', src='file', fpath='data/hospital.csv', embedtxt=False)

INFO:profiler.utility:[0.007339954376220703] Load Data start

INFO:profiler.dataset:inferred types of attributes: {
    "ProviderNumber": "text",
    "HospitalName": "text",
    "Address1": "text",
    "Address2": "categorical",
    "Address3": "categorical",
    "City": "text",
    "State": "categorical",
    "ZipCode": "text",
    "CountyName": "text",
    "PhoneNumber": "text",
    "HospitalType": "text",
    "HospitalOwner": "text",
    "EmergencyService": "categorical",
    "Condition": "text",
    "MeasureCode": "text",
    "MeasureName": "text",
    "Score": "text",
    "Sample": "text",
    "Stateavg": "text"
}
INFO:profiler.dataset:(possible types: numeric, categorical, text)
INFO:profiler.dataset:inferred operators of attributes: {'ProviderNumber': ['equal', 'notequal'], 'HospitalName': ['equal', 'notequal'], 'Address1': ['equal', 'notequal'], 'Address2': ['equal', 'notequal'], 'Address3': ['equal', 'notequal'], 'City': ['equal', 'notequal'], 'State': ['equal', 'notequal'], '

In [4]:
pf.session.ds.change_dtypes(['ProviderNumber', 'ZipCode', 'PhoneNumber', 'State'], 
                         ['categorical', 'numeric', 'categorical', 'text'])

INFO:profiler.dataset:updated types of ProviderNumber to 'categorical'
INFO:profiler.dataset:updated types of ZipCode to 'numeric'
INFO:profiler.dataset:updated types of PhoneNumber to 'categorical'
INFO:profiler.dataset:updated types of State to 'text'
INFO:profiler.dataset:inferred operators of attributes: {'ProviderNumber': ['equal', 'notequal'], 'HospitalName': ['equal', 'notequal'], 'Address1': ['equal', 'notequal'], 'Address2': ['equal', 'notequal'], 'Address3': ['equal', 'notequal'], 'City': ['equal', 'notequal'], 'State': ['equal', 'notequal'], 'ZipCode': ['equal', 'notequal', 'greater_than', 'less_than'], 'CountyName': ['equal', 'notequal'], 'PhoneNumber': ['equal', 'notequal'], 'HospitalType': ['equal', 'notequal'], 'HospitalOwner': ['equal', 'notequal'], 'EmergencyService': ['equal', 'notequal'], 'Condition': ['equal', 'notequal'], 'MeasureCode': ['equal', 'notequal'], 'MeasureName': ['equal', 'notequal'], 'Score': ['equal', 'notequal'], 'Sample': ['equal', 'notequal'], 'Sta

In [5]:
# TODO save embedding vectors
# pf.session.load_embedding()

In [6]:
pf.session.load_training_data()

INFO:profiler.utility:[0.10956192016601562] Create Training Data start

INFO:profiler.utility:[0.25214290618896484] Create Training Data execution time: 0.14258098602294922



In [7]:
pf.session.learn_structure()

INFO:profiler.utility:[0.3608410358428955] Recover Moral Graph start

INFO:profiler.utility:[0.46427297592163086] Recover Moral Graph execution time: 0.10343194007873535

INFO:profiler.utility:[0.46497106552124023] Recover DAG start

INFO:graph:created node ProviderNumber_neq
INFO:graph:created node ProviderNumber_eq
INFO:graph:added directed edge ProviderNumber_neq -> ProviderNumber_eq
INFO:graph:created node HospitalName_eq
INFO:graph:added directed edge ProviderNumber_neq -> HospitalName_eq
INFO:graph:created node HospitalName_neq
INFO:graph:added directed edge ProviderNumber_eq -> HospitalName_neq
INFO:graph:added directed edge HospitalName_eq -> HospitalName_neq
INFO:graph:created node Address1_eq
INFO:graph:added directed edge ProviderNumber_neq -> Address1_eq
INFO:graph:added directed edge HospitalName_neq -> Address1_eq
INFO:graph:created node Address1_neq
INFO:graph:added directed edge ProviderNumber_eq -> Address1_neq
INFO:graph:added directed edge HospitalName_eq -> Address1

INFO:graph:created node Address1_eq
INFO:graph:created node Address1_neq
INFO:graph:created node City_eq
INFO:graph:created node City_neq
INFO:graph:created node State_neq
INFO:graph:created node State_eq
INFO:graph:created node ZipCode_eq
INFO:graph:created node ZipCode_neq
INFO:graph:created node ZipCode_gt
INFO:graph:created node ZipCode_lt
INFO:graph:created node CountyName_eq
INFO:graph:created node CountyName_neq
INFO:graph:created node PhoneNumber_eq
INFO:graph:created node PhoneNumber_neq
INFO:graph:created node HospitalType_neq
INFO:graph:created node HospitalType_eq
INFO:graph:created node HospitalOwner_eq
INFO:graph:created node HospitalOwner_neq
INFO:graph:created node EmergencyService_neq
INFO:graph:created node EmergencyService_eq
INFO:graph:created node Condition_neq
INFO:graph:created node Condition_eq
INFO:graph:created node MeasureCode_eq
INFO:graph:created node MeasureCode_neq
INFO:graph:created node MeasureName_eq
INFO:graph:created node MeasureName_neq
INFO:graph:c

INFO:graph:created node Sample_eq
INFO:graph:created node Sample_neq
INFO:graph:created node Stateavg_eq
INFO:graph:created node Stateavg_neq
INFO:graph:added directed edge ProviderNumber_neq -> Address1_eq
INFO:graph:created node ProviderNumber_neq
INFO:graph:created node ProviderNumber_eq
INFO:graph:created node HospitalName_eq
INFO:graph:created node HospitalName_neq
INFO:graph:created node Address1_eq
INFO:graph:created node Address1_neq
INFO:graph:created node City_eq
INFO:graph:created node City_neq
INFO:graph:created node State_neq
INFO:graph:created node State_eq
INFO:graph:created node ZipCode_eq
INFO:graph:created node ZipCode_neq
INFO:graph:created node ZipCode_gt
INFO:graph:created node ZipCode_lt
INFO:graph:created node CountyName_eq
INFO:graph:created node CountyName_neq
INFO:graph:created node PhoneNumber_eq
INFO:graph:created node PhoneNumber_neq
INFO:graph:created node HospitalType_neq
INFO:graph:created node HospitalType_eq
INFO:graph:created node HospitalOwner_eq
INF

INFO:graph:created node EmergencyService_eq
INFO:graph:created node Condition_neq
INFO:graph:created node Condition_eq
INFO:graph:created node MeasureCode_eq
INFO:graph:created node MeasureCode_neq
INFO:graph:created node MeasureName_eq
INFO:graph:created node MeasureName_neq
INFO:graph:created node Score_neq
INFO:graph:created node Score_eq
INFO:graph:created node Sample_eq
INFO:graph:created node Sample_neq
INFO:graph:created node Stateavg_eq
INFO:graph:created node Stateavg_neq
INFO:graph:created node ProviderNumber_neq
INFO:graph:created node ProviderNumber_eq
INFO:graph:created node HospitalName_eq
INFO:graph:created node HospitalName_neq
INFO:graph:created node Address1_eq
INFO:graph:created node Address1_neq
INFO:graph:created node City_eq
INFO:graph:created node City_neq
INFO:graph:created node State_neq
INFO:graph:created node State_eq
INFO:graph:created node ZipCode_eq
INFO:graph:created node ZipCode_neq
INFO:graph:created node ZipCode_gt
INFO:graph:created node ZipCode_lt
IN

INFO:graph:created node PhoneNumber_eq
INFO:graph:created node PhoneNumber_neq
INFO:graph:created node HospitalType_neq
INFO:graph:created node HospitalType_eq
INFO:graph:created node HospitalOwner_eq
INFO:graph:created node HospitalOwner_neq
INFO:graph:created node EmergencyService_neq
INFO:graph:created node EmergencyService_eq
INFO:graph:created node Condition_neq
INFO:graph:created node Condition_eq
INFO:graph:created node MeasureCode_eq
INFO:graph:created node MeasureCode_neq
INFO:graph:created node MeasureName_eq
INFO:graph:created node MeasureName_neq
INFO:graph:created node Score_neq
INFO:graph:created node Score_eq
INFO:graph:created node Sample_eq
INFO:graph:created node Sample_neq
INFO:graph:created node Stateavg_eq
INFO:graph:created node Stateavg_neq
INFO:graph:added directed edge Condition_neq -> MeasureCode_eq
INFO:graph:created node ProviderNumber_neq
INFO:graph:created node ProviderNumber_eq
INFO:graph:created node HospitalName_eq
INFO:graph:created node HospitalName_n

INFO:graph:created node City_neq
INFO:graph:created node State_neq
INFO:graph:created node State_eq
INFO:graph:created node ZipCode_eq
INFO:graph:created node ZipCode_neq
INFO:graph:created node ZipCode_gt
INFO:graph:created node ZipCode_lt
INFO:graph:created node CountyName_eq
INFO:graph:created node CountyName_neq
INFO:graph:created node PhoneNumber_eq
INFO:graph:created node PhoneNumber_neq
INFO:graph:created node HospitalType_neq
INFO:graph:created node HospitalType_eq
INFO:graph:created node HospitalOwner_eq
INFO:graph:created node HospitalOwner_neq
INFO:graph:created node EmergencyService_neq
INFO:graph:created node EmergencyService_eq
INFO:graph:created node Condition_neq
INFO:graph:created node Condition_eq
INFO:graph:created node MeasureCode_eq
INFO:graph:created node MeasureCode_neq
INFO:graph:created node MeasureName_eq
INFO:graph:created node MeasureName_neq
INFO:graph:created node Score_neq
INFO:graph:created node Score_eq
INFO:graph:created node Sample_eq
INFO:graph:creat

INFO:graph:created node HospitalName_eq
INFO:graph:created node HospitalName_neq
INFO:graph:created node Address1_eq
INFO:graph:created node Address1_neq
INFO:graph:created node City_eq
INFO:graph:created node City_neq
INFO:graph:created node State_neq
INFO:graph:created node State_eq
INFO:graph:created node ZipCode_eq
INFO:graph:created node ZipCode_neq
INFO:graph:created node ZipCode_gt
INFO:graph:created node ZipCode_lt
INFO:graph:created node CountyName_eq
INFO:graph:created node CountyName_neq
INFO:graph:created node PhoneNumber_eq
INFO:graph:created node PhoneNumber_neq
INFO:graph:created node HospitalType_neq
INFO:graph:created node HospitalType_eq
INFO:graph:created node HospitalOwner_eq
INFO:graph:created node HospitalOwner_neq
INFO:graph:created node EmergencyService_neq
INFO:graph:created node EmergencyService_eq
INFO:graph:created node Condition_neq
INFO:graph:created node Condition_eq
INFO:graph:created node MeasureCode_eq
INFO:graph:created node MeasureCode_neq
INFO:graph

In [8]:
s = pf.session.struct_engine

In [12]:
s.decomposed

({15: frozenset({15}),
  16: frozenset({2, 3, 5, 7, 11, 15, 17, 21}),
  18: frozenset({2, 3, 5, 7, 15, 17, 21}),
  22: frozenset({2, 3, 5, 7, 15, 17, 21}),
  2: frozenset({1, 4, 6, 8, 16, 18, 22}),
  3: frozenset({1, 4, 6, 8, 16, 18, 22}),
  4: frozenset({2, 3, 5, 7, 15, 17, 21}),
  5: frozenset({1, 4, 6, 8, 16, 18, 22}),
  6: frozenset({2, 3, 5, 7, 15, 17, 21}),
  7: frozenset({1, 4, 6, 8, 16, 18, 22}),
  8: frozenset({2, 3, 5, 7, 15, 17, 21}),
  17: frozenset({1, 4, 6, 8, 16, 18, 22}),
  21: frozenset({1, 4, 6, 8, 16, 18, 22}),
  1: frozenset({2, 3, 5, 7, 15, 17, 21}),
  12: frozenset({10, 11, 13, 14, 15}),
  36: frozenset({26, 27, 29, 35}),
  30: frozenset({26, 27, 29, 35}),
  29: frozenset({25, 28, 30, 36}),
  35: frozenset({25, 28, 30, 36}),
  25: frozenset({26, 27, 29, 35}),
  26: frozenset({25, 28, 30, 36}),
  27: frozenset({25, 28, 30, 36}),
  28: frozenset({26, 27, 29, 35}),
  11: frozenset({9, 12, 16}),
  13: frozenset({12, 14}),
  10: frozenset({9, 12}),
  9: frozenset({10, 